<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## FINAL PROJECT

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
 
1. <a href="#item30">Data Set-up</a>
2. <a href="#item31">Part A</a>  
3. <a href="#item32">Part B</a>  
4. <a href="#item33">Part C</a>  
5. <a href="#item34">Part D</a>  

</font>
</div>

<a id="item31"></a>

## Data Set-up

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

#### Let's go ahead and import the Keras library

In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import keras

Using TensorFlow backend.
/opt/miniconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/py3/lib/python3.7/site-packag

As you can see, the TensorFlow backend was used to install the Keras library.

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

## Part A

In [43]:
from keras.models import Sequential
from keras.layers import Dense

# define regression model with variable number of hidden layers and nodes
def regression_model(nlayer=1,nnode=10):
    # create model
    model = Sequential()
    model.add(Dense(nnode, activation='relu', input_shape=(n_cols,))) # first hidden layer, 10 nodes
    if nlayer > 1:
        for i in range(nlayer-1):
            model.add(Dense(nnode, activation='relu')) # additionatl hidden layers, 10 nodes
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Next, we will train and test the model at the same time using the *fit* method. We will leave 30% of the original data for testing. We will leave out 30% of the training data for validation and we will train the model for 50 epochs.

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

loss_list = []
for loop in range(50):
    # test-train split
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    # build the model
    model = regression_model(nlayer=1,nnode=10)
    # fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)
    # predict on the testing set
    y_hat = model.predict(X_test)
    # save mse
    mse = mean_squared_error(y_hat,y_test)
    loss_list.append(mse)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 3s 5ms/step - loss: 1542.3598 - val_loss: 1285.6776
Epoch 2/50
504/504 [==============================] - 0s 114us/step - loss: 1094.0547 - val_loss: 931.9951
Epoch 3/50
504/504 [==============================] - 0s 107us/step - loss: 808.3017 - val_loss: 711.3205
Epoch 4/50
504/504 [==============================] - 0s 105us/step - loss: 625.9932 - val_loss: 566.4602
Epoch 5/50
504/504 [==============================] - 0s 114us/step - loss: 505.7493 - val_loss: 476.2895
Epoch 6/50
504/504 [==============================] - 0s 109us/step - loss: 447.7001 - val_loss: 419.3734
Epoch 7/50
504/504 [==============================] - 0s 117us/step - loss: 410.0669 - val_loss: 381.6657
Epoch 8/50
504/504 [==============================] - 0s 109us/step - loss: 383.5375 - val_loss: 351.7347
Epoch 9/50
504/504 [==============================] - 0s 109us/step - loss: 355.3988 - val_loss: 317.6629

504/504 [==============================] - 0s 128us/step - loss: 6938.4752 - val_loss: 7770.2800
Epoch 27/50
504/504 [==============================] - 0s 116us/step - loss: 6745.8216 - val_loss: 7542.4520
Epoch 28/50
504/504 [==============================] - 0s 114us/step - loss: 6542.6146 - val_loss: 7365.6224
Epoch 29/50
504/504 [==============================] - 0s 112us/step - loss: 6358.3030 - val_loss: 7183.8701
Epoch 30/50
504/504 [==============================] - 0s 108us/step - loss: 6181.1118 - val_loss: 6984.6579
Epoch 31/50
504/504 [==============================] - 0s 114us/step - loss: 6001.8226 - val_loss: 6786.9454
Epoch 32/50
504/504 [==============================] - 0s 105us/step - loss: 5832.5983 - val_loss: 6631.5775
Epoch 33/50
504/504 [==============================] - 0s 105us/step - loss: 5664.2462 - val_loss: 6438.0351
Epoch 34/50
504/504 [==============================] - 0s 102us/step - loss: 5503.7640 - val_loss: 6292.0361
Epoch 35/50
504/504 [==========

504/504 [==============================] - 3s 6ms/step - loss: 96183.4802 - val_loss: 59959.9543
Epoch 2/50
504/504 [==============================] - 0s 111us/step - loss: 44896.4901 - val_loss: 25190.8563
Epoch 3/50
504/504 [==============================] - 0s 106us/step - loss: 19691.2824 - val_loss: 12142.6760
Epoch 4/50
504/504 [==============================] - 0s 123us/step - loss: 11481.1854 - val_loss: 9119.3122
Epoch 5/50
504/504 [==============================] - 0s 115us/step - loss: 9398.2824 - val_loss: 8682.2130
Epoch 6/50
504/504 [==============================] - 0s 121us/step - loss: 8967.1836 - val_loss: 8369.3624
Epoch 7/50
504/504 [==============================] - 0s 117us/step - loss: 8642.0697 - val_loss: 8017.7422
Epoch 8/50
504/504 [==============================] - 0s 122us/step - loss: 8267.3469 - val_loss: 7587.1775
Epoch 9/50
504/504 [==============================] - 0s 115us/step - loss: 7917.9630 - val_loss: 7210.9621
Epoch 10/50
504/504 [=============

Epoch 26/50
504/504 [==============================] - 0s 132us/step - loss: 1619.0915 - val_loss: 1419.2596
Epoch 27/50
504/504 [==============================] - 0s 127us/step - loss: 1543.5960 - val_loss: 1354.0279
Epoch 28/50
504/504 [==============================] - 0s 115us/step - loss: 1466.7527 - val_loss: 1294.9819
Epoch 29/50
504/504 [==============================] - 0s 124us/step - loss: 1400.1999 - val_loss: 1235.4656
Epoch 30/50
504/504 [==============================] - 0s 130us/step - loss: 1334.3334 - val_loss: 1183.2848
Epoch 31/50
504/504 [==============================] - 0s 126us/step - loss: 1269.8620 - val_loss: 1116.2452
Epoch 32/50
504/504 [==============================] - 0s 128us/step - loss: 1208.3295 - val_loss: 1072.3792
Epoch 33/50
504/504 [==============================] - 0s 127us/step - loss: 1146.9674 - val_loss: 1007.4561
Epoch 34/50
504/504 [==============================] - 0s 128us/step - loss: 1080.8420 - val_loss: 962.5501
Epoch 35/50
504/504 

504/504 [==============================] - 3s 6ms/step - loss: 3455.3644 - val_loss: 1951.9269
Epoch 2/50
504/504 [==============================] - 0s 120us/step - loss: 1374.9589 - val_loss: 788.9796
Epoch 3/50
504/504 [==============================] - 0s 129us/step - loss: 672.9186 - val_loss: 556.1620
Epoch 4/50
504/504 [==============================] - 0s 113us/step - loss: 559.1806 - val_loss: 548.4228
Epoch 5/50
504/504 [==============================] - 0s 145us/step - loss: 539.8509 - val_loss: 524.5880
Epoch 6/50
504/504 [==============================] - 0s 118us/step - loss: 515.4017 - val_loss: 502.8197
Epoch 7/50
504/504 [==============================] - 0s 115us/step - loss: 488.7985 - val_loss: 475.0127
Epoch 8/50
504/504 [==============================] - 0s 127us/step - loss: 457.9005 - val_loss: 447.2596
Epoch 9/50
504/504 [==============================] - 0s 114us/step - loss: 427.6798 - val_loss: 423.0223
Epoch 10/50
504/504 [==============================] - 0

...

504/504 [==============================] - 0s 135us/step - loss: 143.5980 - val_loss: 155.3793
Epoch 31/50
504/504 [==============================] - 0s 138us/step - loss: 139.8009 - val_loss: 152.3981
Epoch 32/50
504/504 [==============================] - 0s 136us/step - loss: 136.2150 - val_loss: 148.2209
Epoch 33/50
504/504 [==============================] - 0s 138us/step - loss: 133.2141 - val_loss: 145.4231
Epoch 34/50
504/504 [==============================] - 0s 139us/step - loss: 130.3180 - val_loss: 143.0599
Epoch 35/50
504/504 [==============================] - 0s 137us/step - loss: 129.4526 - val_loss: 141.2642
Epoch 36/50
504/504 [==============================] - 0s 135us/step - loss: 126.0226 - val_loss: 137.6182
Epoch 37/50
504/504 [==============================] - 0s 134us/step - loss: 125.0077 - val_loss: 137.0389
Epoch 38/50
504/504 [==============================] - 0s 139us/step - loss: 122.9297 - val_loss: 133.8870
Epoch 39/50
504/504 [============================

Epoch 6/50
504/504 [==============================] - 0s 136us/step - loss: 1315.0378 - val_loss: 1293.2251
Epoch 7/50
504/504 [==============================] - 0s 136us/step - loss: 1231.0372 - val_loss: 1220.7550
Epoch 8/50
504/504 [==============================] - 0s 136us/step - loss: 1168.3714 - val_loss: 1154.0374
Epoch 9/50
504/504 [==============================] - 0s 132us/step - loss: 1107.2291 - val_loss: 1092.7680
Epoch 10/50
504/504 [==============================] - 0s 133us/step - loss: 1055.2591 - val_loss: 1036.2620
Epoch 11/50
504/504 [==============================] - 0s 136us/step - loss: 998.1883 - val_loss: 970.1635
Epoch 12/50
504/504 [==============================] - 0s 134us/step - loss: 938.2525 - val_loss: 924.4730
Epoch 13/50
504/504 [==============================] - 0s 135us/step - loss: 892.9772 - val_loss: 872.4325
Epoch 14/50
504/504 [==============================] - 0s 136us/step - loss: 841.6802 - val_loss: 820.6011
Epoch 15/50
504/504 [==========

504/504 [==============================] - 0s 138us/step - loss: 1253.2559 - val_loss: 1415.8468
Epoch 32/50
504/504 [==============================] - 0s 139us/step - loss: 1231.2058 - val_loss: 1396.0234
Epoch 33/50
504/504 [==============================] - 0s 138us/step - loss: 1211.7234 - val_loss: 1376.7103
Epoch 34/50
504/504 [==============================] - 0s 139us/step - loss: 1191.5138 - val_loss: 1354.8277
Epoch 35/50
504/504 [==============================] - 0s 134us/step - loss: 1173.7589 - val_loss: 1334.0887
Epoch 36/50
504/504 [==============================] - 0s 136us/step - loss: 1156.0949 - val_loss: 1313.4845
Epoch 37/50
504/504 [==============================] - 0s 139us/step - loss: 1138.4462 - val_loss: 1294.2220
Epoch 38/50
504/504 [==============================] - 0s 156us/step - loss: 1121.6303 - val_loss: 1274.5050
Epoch 39/50
504/504 [==============================] - 0s 137us/step - loss: 1103.9358 - val_loss: 1255.0380
Epoch 40/50
504/504 [==========

504/504 [==============================] - 0s 138us/step - loss: 103907.8320 - val_loss: 92017.8061
Epoch 7/50
504/504 [==============================] - 0s 134us/step - loss: 85678.0601 - val_loss: 76316.9997
Epoch 8/50
504/504 [==============================] - 0s 140us/step - loss: 71378.2055 - val_loss: 63762.5417
Epoch 9/50
504/504 [==============================] - 0s 136us/step - loss: 59825.6146 - val_loss: 53661.0834
Epoch 10/50
504/504 [==============================] - 0s 151us/step - loss: 50480.5840 - val_loss: 45415.6601
Epoch 11/50
504/504 [==============================] - 0s 140us/step - loss: 42848.1006 - val_loss: 38805.3834
Epoch 12/50
504/504 [==============================] - 0s 156us/step - loss: 36715.5273 - val_loss: 33500.2559
Epoch 13/50
504/504 [==============================] - 0s 146us/step - loss: 31761.9747 - val_loss: 29127.0569
Epoch 14/50
504/504 [==============================] - 0s 143us/step - loss: 27689.9346 - val_loss: 25514.7282
Epoch 15/50
504

504/504 [==============================] - 0s 140us/step - loss: 171.8413 - val_loss: 123.7750
Epoch 32/50
504/504 [==============================] - 0s 139us/step - loss: 167.5182 - val_loss: 120.9348
Epoch 33/50
504/504 [==============================] - 0s 136us/step - loss: 162.3757 - val_loss: 119.8552
Epoch 34/50
504/504 [==============================] - 0s 141us/step - loss: 158.9423 - val_loss: 116.3979
Epoch 35/50
504/504 [==============================] - 0s 159us/step - loss: 154.2741 - val_loss: 115.4746
Epoch 36/50
504/504 [==============================] - 0s 130us/step - loss: 151.9782 - val_loss: 112.9035
Epoch 37/50
504/504 [==============================] - 0s 141us/step - loss: 149.0305 - val_loss: 110.8333
Epoch 38/50
504/504 [==============================] - 0s 141us/step - loss: 145.8981 - val_loss: 109.4683
Epoch 39/50
504/504 [==============================] - 0s 154us/step - loss: 142.6132 - val_loss: 108.0618
Epoch 40/50
504/504 [============================

In [45]:
import numpy as np
print('The mean MSE is',np.mean(loss_list))
print('The standard deviation of MSE is',np.std(loss_list))

The mean MSE is 611.4423077087941
The standard deviation of MSE is 816.8483483836516


## Part B

In [38]:
loss_list = []
for loop in range(50):
    # test-train split using normalized data
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    # build the model
    model = regression_model(nlayer=1,nnode=10)
    # fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)
    # predict on the testing set
    y_hat = model.predict(X_test)
    # save mse
    mse = mean_squared_error(y_hat,y_test)
    loss_list.append(mse)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 2s 3ms/step - loss: 1590.4763 - val_loss: 1476.7342
Epoch 2/50
504/504 [==============================] - 0s 86us/step - loss: 1564.8479 - val_loss: 1453.7565
Epoch 3/50
504/504 [==============================] - 0s 83us/step - loss: 1539.7818 - val_loss: 1430.9523
Epoch 4/50
504/504 [==============================] - 0s 81us/step - loss: 1514.6419 - val_loss: 1407.5719
Epoch 5/50
504/504 [==============================] - 0s 88us/step - loss: 1488.6535 - val_loss: 1383.0244
Epoch 6/50
504/504 [==============================] - 0s 89us/step - loss: 1461.2606 - val_loss: 1357.0926
Epoch 7/50
504/504 [==============================] - 0s 87us/step - loss: 1431.7999 - val_loss: 1329.2134
Epoch 8/50
504/504 [==============================] - 0s 88us/step - loss: 1400.0565 - val_loss: 1298.8750
Epoch 9/50
504/504 [==============================] - 0s 89us/step - loss: 1365.3086 - val_loss: 12

504/504 [==============================] - 0s 88us/step - loss: 451.5726 - val_loss: 437.2582
Epoch 28/50
504/504 [==============================] - 0s 87us/step - loss: 416.4572 - val_loss: 405.2933
Epoch 29/50
504/504 [==============================] - 0s 91us/step - loss: 384.0212 - val_loss: 377.0880
Epoch 30/50
504/504 [==============================] - 0s 88us/step - loss: 355.7393 - val_loss: 351.2961
Epoch 31/50
504/504 [==============================] - 0s 87us/step - loss: 329.9980 - val_loss: 328.7152
Epoch 32/50
504/504 [==============================] - 0s 89us/step - loss: 308.0192 - val_loss: 308.0727
Epoch 33/50
504/504 [==============================] - 0s 91us/step - loss: 288.1874 - val_loss: 290.5190
Epoch 34/50
504/504 [==============================] - 0s 91us/step - loss: 270.4185 - val_loss: 276.3086
Epoch 35/50
504/504 [==============================] - 0s 92us/step - loss: 256.4671 - val_loss: 263.2297
Epoch 36/50
504/504 [==============================] - 0s 

504/504 [==============================] - 0s 81us/step - loss: 1487.4630 - val_loss: 1524.4466
Epoch 4/50
504/504 [==============================] - 0s 94us/step - loss: 1460.6999 - val_loss: 1495.5065
Epoch 5/50
504/504 [==============================] - 0s 90us/step - loss: 1432.9001 - val_loss: 1465.6761
Epoch 6/50
504/504 [==============================] - 0s 89us/step - loss: 1403.4079 - val_loss: 1434.6529
Epoch 7/50
504/504 [==============================] - 0s 87us/step - loss: 1372.9065 - val_loss: 1400.9328
Epoch 8/50
504/504 [==============================] - 0s 82us/step - loss: 1340.0934 - val_loss: 1365.2851
Epoch 9/50
504/504 [==============================] - 0s 84us/step - loss: 1305.0851 - val_loss: 1327.6358
Epoch 10/50
504/504 [==============================] - 0s 82us/step - loss: 1268.7673 - val_loss: 1286.9468
Epoch 11/50
504/504 [==============================] - 0s 82us/step - loss: 1229.3531 - val_loss: 1245.3087
Epoch 12/50
504/504 [=========================

504/504 [==============================] - 0s 91us/step - loss: 372.3898 - val_loss: 303.2480
Epoch 30/50
504/504 [==============================] - 0s 86us/step - loss: 346.4599 - val_loss: 281.4488
Epoch 31/50
504/504 [==============================] - 0s 83us/step - loss: 323.8132 - val_loss: 262.4337
Epoch 32/50
504/504 [==============================] - 0s 84us/step - loss: 304.3946 - val_loss: 245.6883
Epoch 33/50
504/504 [==============================] - 0s 85us/step - loss: 287.6251 - val_loss: 230.9469
Epoch 34/50
504/504 [==============================] - 0s 98us/step - loss: 272.6020 - val_loss: 218.8935
Epoch 35/50
504/504 [==============================] - 0s 96us/step - loss: 259.8716 - val_loss: 208.8005
Epoch 36/50
504/504 [==============================] - 0s 84us/step - loss: 248.5881 - val_loss: 200.6356
Epoch 37/50
504/504 [==============================] - 0s 88us/step - loss: 239.5372 - val_loss: 193.4044
Epoch 38/50
504/504 [==============================] - 0s 

504/504 [==============================] - 0s 85us/step - loss: 1411.1457 - val_loss: 1505.7383
Epoch 6/50
504/504 [==============================] - 0s 85us/step - loss: 1380.9445 - val_loss: 1473.1782
Epoch 7/50
504/504 [==============================] - 0s 85us/step - loss: 1348.5602 - val_loss: 1438.6781
Epoch 8/50
504/504 [==============================] - 0s 89us/step - loss: 1314.0068 - val_loss: 1401.2384
Epoch 9/50
504/504 [==============================] - 0s 85us/step - loss: 1276.1696 - val_loss: 1361.6024
Epoch 10/50
504/504 [==============================] - 0s 83us/step - loss: 1236.0569 - val_loss: 1318.9609
Epoch 11/50
504/504 [==============================] - 0s 89us/step - loss: 1193.6350 - val_loss: 1272.5535
Epoch 12/50
504/504 [==============================] - 0s 93us/step - loss: 1147.2184 - val_loss: 1224.9497
Epoch 13/50
504/504 [==============================] - 0s 89us/step - loss: 1099.5545 - val_loss: 1174.2870
Epoch 14/50
504/504 [=======================

...

504/504 [==============================] - 0s 109us/step - loss: 577.9567 - val_loss: 645.3036
Epoch 24/50
504/504 [==============================] - 0s 112us/step - loss: 534.4365 - val_loss: 599.8401
Epoch 25/50
504/504 [==============================] - 0s 109us/step - loss: 493.8407 - val_loss: 556.8073
Epoch 26/50
504/504 [==============================] - 0s 108us/step - loss: 456.0813 - val_loss: 517.1108
Epoch 27/50
504/504 [==============================] - 0s 106us/step - loss: 421.6730 - val_loss: 480.2981
Epoch 28/50
504/504 [==============================] - 0s 112us/step - loss: 390.2718 - val_loss: 447.0999
Epoch 29/50
504/504 [==============================] - 0s 110us/step - loss: 362.1748 - val_loss: 417.3437
Epoch 30/50
504/504 [==============================] - 0s 114us/step - loss: 337.9491 - val_loss: 389.9186
Epoch 31/50
504/504 [==============================] - 0s 114us/step - loss: 315.5742 - val_loss: 366.8110
Epoch 32/50
504/504 [============================

504/504 [==============================] - 0s 116us/step - loss: 178.3970 - val_loss: 183.2286
Epoch 50/50
504/504 [==============================] - 0s 104us/step - loss: 175.8654 - val_loss: 181.5476
Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 2s 5ms/step - loss: 1610.5224 - val_loss: 1547.9017
Epoch 2/50
504/504 [==============================] - 0s 105us/step - loss: 1580.5311 - val_loss: 1520.8723
Epoch 3/50
504/504 [==============================] - 0s 105us/step - loss: 1551.2622 - val_loss: 1494.0760
Epoch 4/50
504/504 [==============================] - 0s 104us/step - loss: 1522.3450 - val_loss: 1467.0660
Epoch 5/50
504/504 [==============================] - 0s 101us/step - loss: 1493.0306 - val_loss: 1439.2159
Epoch 6/50
504/504 [==============================] - 0s 115us/step - loss: 1462.1419 - val_loss: 1410.9821
Epoch 7/50
504/504 [==============================] - 0s 102us/step - loss: 1430.5532 - val_loss: 1380.963

504/504 [==============================] - 0s 121us/step - loss: 564.6820 - val_loss: 585.5978
Epoch 25/50
504/504 [==============================] - 0s 113us/step - loss: 520.3935 - val_loss: 539.6835
Epoch 26/50
504/504 [==============================] - 0s 113us/step - loss: 479.4662 - val_loss: 497.1026
Epoch 27/50
504/504 [==============================] - 0s 114us/step - loss: 442.3824 - val_loss: 456.7938
Epoch 28/50
504/504 [==============================] - 0s 109us/step - loss: 408.2363 - val_loss: 420.6447
Epoch 29/50
504/504 [==============================] - 0s 128us/step - loss: 376.9402 - val_loss: 389.7388
Epoch 30/50
504/504 [==============================] - 0s 128us/step - loss: 350.4860 - val_loss: 360.6474
Epoch 31/50
504/504 [==============================] - 0s 117us/step - loss: 326.7762 - val_loss: 334.9366
Epoch 32/50
504/504 [==============================] - 0s 112us/step - loss: 305.8620 - val_loss: 312.0587
Epoch 33/50
504/504 [============================

504/504 [==============================] - 0s 112us/step - loss: 193.7688 - val_loss: 217.1140
Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 3s 5ms/step - loss: 1517.7936 - val_loss: 1614.0420
Epoch 2/50
504/504 [==============================] - 0s 123us/step - loss: 1491.8088 - val_loss: 1586.5343
Epoch 3/50
504/504 [==============================] - 0s 120us/step - loss: 1464.9014 - val_loss: 1558.1506
Epoch 4/50
504/504 [==============================] - 0s 114us/step - loss: 1436.8849 - val_loss: 1528.2658
Epoch 5/50
504/504 [==============================] - 0s 108us/step - loss: 1406.6736 - val_loss: 1496.2152
Epoch 6/50
504/504 [==============================] - 0s 113us/step - loss: 1374.8208 - val_loss: 1461.2394
Epoch 7/50
504/504 [==============================] - 0s 109us/step - loss: 1339.2785 - val_loss: 1423.9981
Epoch 8/50
504/504 [==============================] - 0s 115us/step - loss: 1302.0864 - val_loss: 1382.67

504/504 [==============================] - 0s 108us/step - loss: 502.2605 - val_loss: 493.0001
Epoch 26/50
504/504 [==============================] - 0s 121us/step - loss: 461.7913 - val_loss: 456.3293
Epoch 27/50
504/504 [==============================] - 0s 123us/step - loss: 424.8104 - val_loss: 422.9699
Epoch 28/50
504/504 [==============================] - 0s 127us/step - loss: 391.3650 - val_loss: 392.5365
Epoch 29/50
504/504 [==============================] - 0s 111us/step - loss: 361.8426 - val_loss: 365.2558
Epoch 30/50
504/504 [==============================] - 0s 117us/step - loss: 335.1833 - val_loss: 341.7940
Epoch 31/50
504/504 [==============================] - 0s 113us/step - loss: 311.8916 - val_loss: 321.6251
Epoch 32/50
504/504 [==============================] - 0s 117us/step - loss: 292.4585 - val_loss: 303.0842
Epoch 33/50
504/504 [==============================] - 0s 113us/step - loss: 274.6545 - val_loss: 288.0710
Epoch 34/50
504/504 [============================

In [42]:
import numpy as np
print('The mean MSE is',np.mean(loss_list))
print('The standard deviation of MSE is',np.std(loss_list))

The mean MSE is 187.5525390307587
The standard deviation of MSE is 16.90021290018166


MSE has decreased a lot by using normalized data. It appears to be better to use the noralized data.

## Part C

In [50]:
loss_list = []
for loop in range(50):
    # test-train split using normalized data
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    # build the model
    model = regression_model(nlayer=1,nnode=10)
    # fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=1) # 100 epochs
    # predict on the testing set
    y_hat = model.predict(X_test)
    # save mse
    mse = mean_squared_error(y_hat,y_test)
    loss_list.append(mse)

Train on 504 samples, validate on 217 samples
Epoch 1/100
504/504 [==============================] - 7s 13ms/step - loss: 1487.6700 - val_loss: 1564.9506
Epoch 2/100
504/504 [==============================] - 0s 271us/step - loss: 1476.5489 - val_loss: 1554.2478
Epoch 3/100
504/504 [==============================] - 0s 255us/step - loss: 1465.2055 - val_loss: 1543.2948
Epoch 4/100
504/504 [==============================] - 0s 244us/step - loss: 1453.5911 - val_loss: 1532.0138
Epoch 5/100
504/504 [==============================] - 0s 256us/step - loss: 1441.7892 - val_loss: 1520.1668
Epoch 6/100
504/504 [==============================] - 0s 250us/step - loss: 1429.4596 - val_loss: 1508.1390
Epoch 7/100
504/504 [==============================] - 0s 250us/step - loss: 1416.9096 - val_loss: 1495.7349
Epoch 8/100
504/504 [==============================] - 0s 258us/step - loss: 1403.8815 - val_loss: 1483.0448
Epoch 9/100
504/504 [==============================] - 0s 253us/step - loss: 1390.6

Epoch 76/100
504/504 [==============================] - 0s 266us/step - loss: 290.2532 - val_loss: 311.2749
Epoch 77/100
504/504 [==============================] - 0s 275us/step - loss: 282.2604 - val_loss: 302.7840
Epoch 78/100
504/504 [==============================] - 0s 263us/step - loss: 274.8813 - val_loss: 294.8128
Epoch 79/100
504/504 [==============================] - 0s 266us/step - loss: 267.8670 - val_loss: 287.0708
Epoch 80/100
504/504 [==============================] - 0s 277us/step - loss: 261.4005 - val_loss: 279.6050
Epoch 81/100
504/504 [==============================] - 0s 273us/step - loss: 255.0343 - val_loss: 272.6888
Epoch 82/100
504/504 [==============================] - 0s 278us/step - loss: 249.1675 - val_loss: 266.1725
Epoch 83/100
504/504 [==============================] - 0s 273us/step - loss: 243.5177 - val_loss: 260.1252
Epoch 84/100
504/504 [==============================] - 0s 272us/step - loss: 238.3137 - val_loss: 254.1237
Epoch 85/100
504/504 [======

Epoch 51/100
504/504 [==============================] - 0s 257us/step - loss: 646.5854 - val_loss: 646.6923
Epoch 52/100
504/504 [==============================] - 0s 268us/step - loss: 628.7423 - val_loss: 628.8687
Epoch 53/100
504/504 [==============================] - 0s 276us/step - loss: 610.8405 - val_loss: 611.8376
Epoch 54/100
504/504 [==============================] - 0s 265us/step - loss: 593.2590 - val_loss: 595.2630
Epoch 55/100
504/504 [==============================] - 0s 269us/step - loss: 576.4641 - val_loss: 579.2297
Epoch 56/100
504/504 [==============================] - 0s 276us/step - loss: 560.0241 - val_loss: 563.5774
Epoch 57/100
504/504 [==============================] - 0s 278us/step - loss: 543.6749 - val_loss: 548.5207
Epoch 58/100
504/504 [==============================] - 0s 262us/step - loss: 528.3400 - val_loss: 533.5562
Epoch 59/100
504/504 [==============================] - 0s 258us/step - loss: 513.4643 - val_loss: 518.8567
Epoch 60/100
504/504 [======

504/504 [==============================] - 0s 263us/step - loss: 1221.6386 - val_loss: 1246.8809
Epoch 27/100
504/504 [==============================] - 0s 266us/step - loss: 1203.1266 - val_loss: 1229.0812
Epoch 28/100
504/504 [==============================] - 0s 270us/step - loss: 1184.8082 - val_loss: 1210.5144
Epoch 29/100
504/504 [==============================] - 0s 255us/step - loss: 1165.6759 - val_loss: 1191.8291
Epoch 30/100
504/504 [==============================] - 0s 269us/step - loss: 1145.9509 - val_loss: 1173.2783
Epoch 31/100
504/504 [==============================] - 0s 272us/step - loss: 1126.6518 - val_loss: 1153.8346
Epoch 32/100
504/504 [==============================] - 0s 273us/step - loss: 1106.7293 - val_loss: 1134.3341
Epoch 33/100
504/504 [==============================] - 0s 265us/step - loss: 1086.3694 - val_loss: 1115.0296
Epoch 34/100
504/504 [==============================] - 0s 267us/step - loss: 1066.5590 - val_loss: 1095.0900
Epoch 35/100
504/504 [=

504/504 [==============================] - 7s 14ms/step - loss: 1560.5911 - val_loss: 1813.4281
Epoch 2/100
504/504 [==============================] - 0s 273us/step - loss: 1547.6367 - val_loss: 1798.8158
Epoch 3/100
504/504 [==============================] - 0s 260us/step - loss: 1535.3205 - val_loss: 1784.5931
Epoch 4/100
504/504 [==============================] - 0s 281us/step - loss: 1523.2112 - val_loss: 1771.0888
Epoch 5/100
504/504 [==============================] - 0s 275us/step - loss: 1511.5995 - val_loss: 1757.9852
Epoch 6/100
504/504 [==============================] - 0s 286us/step - loss: 1500.4252 - val_loss: 1744.9811
Epoch 7/100
504/504 [==============================] - 0s 272us/step - loss: 1489.2900 - val_loss: 1732.4115
Epoch 8/100
504/504 [==============================] - 0s 274us/step - loss: 1478.3567 - val_loss: 1720.0028
Epoch 9/100
504/504 [==============================] - 0s 287us/step - loss: 1467.6228 - val_loss: 1707.6066
Epoch 10/100
504/504 [==========

...

504/504 [==============================] - 0s 313us/step - loss: 592.6206 - val_loss: 591.5389
Epoch 52/100
504/504 [==============================] - 0s 336us/step - loss: 573.3134 - val_loss: 572.4470
Epoch 53/100
504/504 [==============================] - 0s 306us/step - loss: 554.2100 - val_loss: 554.4881
Epoch 54/100
504/504 [==============================] - 0s 334us/step - loss: 536.2341 - val_loss: 536.5050
Epoch 55/100
504/504 [==============================] - 0s 336us/step - loss: 518.7782 - val_loss: 518.9503
Epoch 56/100
504/504 [==============================] - 0s 330us/step - loss: 501.5489 - val_loss: 502.3640
Epoch 57/100
504/504 [==============================] - 0s 336us/step - loss: 485.1520 - val_loss: 486.4352
Epoch 58/100
504/504 [==============================] - 0s 339us/step - loss: 469.5668 - val_loss: 470.7591
Epoch 59/100
504/504 [==============================] - 0s 339us/step - loss: 454.5676 - val_loss: 455.5662
Epoch 60/100
504/504 [===================

504/504 [==============================] - 0s 301us/step - loss: 1185.0682 - val_loss: 1052.1475
Epoch 27/100
504/504 [==============================] - 0s 301us/step - loss: 1164.0846 - val_loss: 1033.7106
Epoch 28/100
504/504 [==============================] - 0s 308us/step - loss: 1143.4491 - val_loss: 1014.0181
Epoch 29/100
504/504 [==============================] - 0s 338us/step - loss: 1121.5231 - val_loss: 994.6760
Epoch 30/100
504/504 [==============================] - 0s 333us/step - loss: 1099.3986 - val_loss: 975.1058
Epoch 31/100
504/504 [==============================] - 0s 335us/step - loss: 1077.0834 - val_loss: 955.3318
Epoch 32/100
504/504 [==============================] - 0s 332us/step - loss: 1054.5972 - val_loss: 934.9571
Epoch 33/100
504/504 [==============================] - 0s 338us/step - loss: 1031.3774 - val_loss: 914.9243
Epoch 34/100
504/504 [==============================] - 0s 344us/step - loss: 1008.3792 - val_loss: 894.4081
Epoch 35/100
504/504 [=======

504/504 [==============================] - 8s 16ms/step - loss: 1483.3176 - val_loss: 1482.7820
Epoch 2/100
504/504 [==============================] - 0s 334us/step - loss: 1472.1353 - val_loss: 1472.1149
Epoch 3/100
504/504 [==============================] - 0s 338us/step - loss: 1460.6335 - val_loss: 1460.9570
Epoch 4/100
504/504 [==============================] - 0s 326us/step - loss: 1448.7242 - val_loss: 1449.4226
Epoch 5/100
504/504 [==============================] - 0s 314us/step - loss: 1436.4685 - val_loss: 1437.2825
Epoch 6/100
504/504 [==============================] - 0s 335us/step - loss: 1423.7947 - val_loss: 1424.5889
Epoch 7/100
504/504 [==============================] - 0s 302us/step - loss: 1410.3505 - val_loss: 1411.4947
Epoch 8/100
504/504 [==============================] - 0s 329us/step - loss: 1396.5554 - val_loss: 1397.7345
Epoch 9/100
504/504 [==============================] - 0s 314us/step - loss: 1382.0721 - val_loss: 1383.3023
Epoch 10/100
504/504 [==========

504/504 [==============================] - 0s 332us/step - loss: 262.4453 - val_loss: 255.5690
Epoch 77/100
504/504 [==============================] - 0s 318us/step - loss: 258.7466 - val_loss: 250.9960
Epoch 78/100
504/504 [==============================] - 0s 321us/step - loss: 255.0751 - val_loss: 246.8446
Epoch 79/100
504/504 [==============================] - 0s 312us/step - loss: 251.4840 - val_loss: 242.9801
Epoch 80/100
504/504 [==============================] - 0s 332us/step - loss: 248.2305 - val_loss: 239.1679
Epoch 81/100
504/504 [==============================] - 0s 320us/step - loss: 245.0945 - val_loss: 235.4371
Epoch 82/100
504/504 [==============================] - 0s 332us/step - loss: 241.9886 - val_loss: 231.9654
Epoch 83/100
504/504 [==============================] - 0s 306us/step - loss: 239.1135 - val_loss: 228.5446
Epoch 84/100
504/504 [==============================] - 0s 334us/step - loss: 236.2779 - val_loss: 225.5042
Epoch 85/100
504/504 [===================

504/504 [==============================] - 0s 338us/step - loss: 521.8144 - val_loss: 468.8879
Epoch 52/100
504/504 [==============================] - 0s 307us/step - loss: 503.6412 - val_loss: 452.6051
Epoch 53/100
504/504 [==============================] - 0s 307us/step - loss: 486.2297 - val_loss: 437.2128
Epoch 54/100
504/504 [==============================] - 0s 329us/step - loss: 469.4455 - val_loss: 422.3585
Epoch 55/100
504/504 [==============================] - 0s 343us/step - loss: 453.3105 - val_loss: 408.1479
Epoch 56/100
504/504 [==============================] - 0s 327us/step - loss: 437.8484 - val_loss: 394.4043
Epoch 57/100
504/504 [==============================] - 0s 332us/step - loss: 422.4982 - val_loss: 381.5825
Epoch 58/100
504/504 [==============================] - 0s 347us/step - loss: 408.4804 - val_loss: 368.9805
Epoch 59/100
504/504 [==============================] - 0s 324us/step - loss: 394.2461 - val_loss: 357.7078
Epoch 60/100
504/504 [===================

In [51]:
print('The mean MSE is',np.mean(loss_list))
print('The standard deviation of MSE is',np.std(loss_list))

The mean MSE is 223.0895737265187
The standard deviation of MSE is 45.74265738261462


Using 100 epochs didn't really help. The mean MSE increased slightly.

## Part D

In [48]:
loss_list = []
for loop in range(50):
    # test-train split using normalized data
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3) # use normalized data
    # build the model
    model = regression_model(nlayer=3,nnode=10)
    # fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)
    # predict on the testing set
    y_hat = model.predict(X_test)
    # save mse
    mse = mean_squared_error(y_hat,y_test)
    loss_list.append(mse)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 5s 10ms/step - loss: 1538.6664 - val_loss: 1660.2484
Epoch 2/50
504/504 [==============================] - 0s 218us/step - loss: 1532.2117 - val_loss: 1652.5828
Epoch 3/50
504/504 [==============================] - 0s 203us/step - loss: 1524.7273 - val_loss: 1641.9302
Epoch 4/50
504/504 [==============================] - 0s 214us/step - loss: 1513.1123 - val_loss: 1624.3745
Epoch 5/50
504/504 [==============================] - 0s 220us/step - loss: 1492.9800 - val_loss: 1595.8862
Epoch 6/50
504/504 [==============================] - 0s 208us/step - loss: 1461.0762 - val_loss: 1553.2612
Epoch 7/50
504/504 [==============================] - 0s 217us/step - loss: 1415.4024 - val_loss: 1494.0701
Epoch 8/50
504/504 [==============================] - 0s 209us/step - loss: 1354.0260 - val_loss: 1409.5715
Epoch 9/50
504/504 [==============================] - 0s 214us/step - loss: 1266.1698 - val

504/504 [==============================] - 0s 217us/step - loss: 176.5916 - val_loss: 188.8894
Epoch 27/50
504/504 [==============================] - 0s 226us/step - loss: 172.8802 - val_loss: 187.1331
Epoch 28/50
504/504 [==============================] - 0s 217us/step - loss: 169.8745 - val_loss: 185.3119
Epoch 29/50
504/504 [==============================] - 0s 220us/step - loss: 166.6723 - val_loss: 183.1749
Epoch 30/50
504/504 [==============================] - 0s 216us/step - loss: 163.8627 - val_loss: 181.5609
Epoch 31/50
504/504 [==============================] - 0s 224us/step - loss: 161.3308 - val_loss: 179.9915
Epoch 32/50
504/504 [==============================] - 0s 224us/step - loss: 159.0786 - val_loss: 177.9739
Epoch 33/50
504/504 [==============================] - 0s 225us/step - loss: 156.5973 - val_loss: 176.4778
Epoch 34/50
504/504 [==============================] - 0s 215us/step - loss: 154.7873 - val_loss: 174.9026
Epoch 35/50
504/504 [============================

504/504 [==============================] - 5s 10ms/step - loss: 1544.2580 - val_loss: 1649.3159
Epoch 2/50
504/504 [==============================] - 0s 219us/step - loss: 1530.4513 - val_loss: 1632.4114
Epoch 3/50
504/504 [==============================] - 0s 219us/step - loss: 1512.6529 - val_loss: 1607.2634
Epoch 4/50
504/504 [==============================] - 0s 220us/step - loss: 1486.3280 - val_loss: 1572.5352
Epoch 5/50
504/504 [==============================] - 0s 215us/step - loss: 1448.3180 - val_loss: 1524.9168
Epoch 6/50
504/504 [==============================] - 0s 229us/step - loss: 1396.0469 - val_loss: 1457.9442
Epoch 7/50
504/504 [==============================] - 0s 221us/step - loss: 1324.5437 - val_loss: 1373.5252
Epoch 8/50
504/504 [==============================] - 0s 223us/step - loss: 1238.3595 - val_loss: 1270.7233
Epoch 9/50
504/504 [==============================] - 0s 217us/step - loss: 1135.5706 - val_loss: 1150.3486
Epoch 10/50
504/504 [===================

504/504 [==============================] - 0s 229us/step - loss: 189.7636 - val_loss: 216.7254
Epoch 28/50
504/504 [==============================] - 0s 229us/step - loss: 186.2129 - val_loss: 212.7431
Epoch 29/50
504/504 [==============================] - 0s 226us/step - loss: 182.5003 - val_loss: 208.3581
Epoch 30/50
504/504 [==============================] - 0s 231us/step - loss: 179.7032 - val_loss: 204.6406
Epoch 31/50
504/504 [==============================] - 0s 234us/step - loss: 176.5434 - val_loss: 200.8469
Epoch 32/50
504/504 [==============================] - 0s 230us/step - loss: 173.5155 - val_loss: 197.2027
Epoch 33/50
504/504 [==============================] - 0s 237us/step - loss: 170.7029 - val_loss: 193.9972
Epoch 34/50
504/504 [==============================] - 0s 225us/step - loss: 168.0977 - val_loss: 191.0840
Epoch 35/50
504/504 [==============================] - 0s 227us/step - loss: 165.9109 - val_loss: 187.1970
Epoch 36/50
504/504 [============================

504/504 [==============================] - 0s 235us/step - loss: 1543.3178 - val_loss: 1417.5550
Epoch 3/50
504/504 [==============================] - 0s 231us/step - loss: 1520.7953 - val_loss: 1392.0299
Epoch 4/50
504/504 [==============================] - 0s 235us/step - loss: 1492.5236 - val_loss: 1359.3171
Epoch 5/50
504/504 [==============================] - 0s 229us/step - loss: 1454.1170 - val_loss: 1316.2508
Epoch 6/50
504/504 [==============================] - 0s 236us/step - loss: 1404.3351 - val_loss: 1257.8277
Epoch 7/50
504/504 [==============================] - 0s 228us/step - loss: 1336.6247 - val_loss: 1180.6624
Epoch 8/50
504/504 [==============================] - 0s 235us/step - loss: 1245.2900 - val_loss: 1073.3348
Epoch 9/50
504/504 [==============================] - 0s 230us/step - loss: 1117.9417 - val_loss: 934.3940
Epoch 10/50
504/504 [==============================] - 0s 220us/step - loss: 959.5756 - val_loss: 772.2652
Epoch 11/50
504/504 [====================

...

504/504 [==============================] - 0s 313us/step - loss: 155.4413 - val_loss: 197.4759
Epoch 41/50
504/504 [==============================] - 0s 294us/step - loss: 154.0459 - val_loss: 195.3047
Epoch 42/50
504/504 [==============================] - 0s 291us/step - loss: 152.4797 - val_loss: 192.1621
Epoch 43/50
504/504 [==============================] - 0s 294us/step - loss: 151.3290 - val_loss: 191.6954
Epoch 44/50
504/504 [==============================] - 0s 294us/step - loss: 149.9166 - val_loss: 189.9729
Epoch 45/50
504/504 [==============================] - 0s 290us/step - loss: 149.0899 - val_loss: 188.4287
Epoch 46/50
504/504 [==============================] - 0s 294us/step - loss: 147.6046 - val_loss: 186.1219
Epoch 47/50
504/504 [==============================] - 0s 297us/step - loss: 146.6396 - val_loss: 185.1663
Epoch 48/50
504/504 [==============================] - 0s 294us/step - loss: 145.4378 - val_loss: 183.3489
Epoch 49/50
504/504 [============================

504/504 [==============================] - 0s 297us/step - loss: 647.6544 - val_loss: 620.9875
Epoch 16/50
504/504 [==============================] - 0s 294us/step - loss: 509.2587 - val_loss: 488.4413
Epoch 17/50
504/504 [==============================] - 0s 294us/step - loss: 398.1332 - val_loss: 391.5442
Epoch 18/50
504/504 [==============================] - 0s 293us/step - loss: 322.8418 - val_loss: 331.1159
Epoch 19/50
504/504 [==============================] - 0s 296us/step - loss: 276.5156 - val_loss: 298.0344
Epoch 20/50
504/504 [==============================] - 0s 293us/step - loss: 252.1015 - val_loss: 275.7188
Epoch 21/50
504/504 [==============================] - 0s 291us/step - loss: 236.5594 - val_loss: 259.5038
Epoch 22/50
504/504 [==============================] - 0s 296us/step - loss: 224.4519 - val_loss: 246.0927
Epoch 23/50
504/504 [==============================] - 0s 290us/step - loss: 214.2362 - val_loss: 236.0496
Epoch 24/50
504/504 [============================

504/504 [==============================] - 0s 309us/step - loss: 136.3948 - val_loss: 167.6067
Epoch 42/50
504/504 [==============================] - 0s 298us/step - loss: 134.7965 - val_loss: 167.4905
Epoch 43/50
504/504 [==============================] - 0s 318us/step - loss: 132.7112 - val_loss: 165.3469
Epoch 44/50
504/504 [==============================] - 0s 291us/step - loss: 130.7988 - val_loss: 163.8214
Epoch 45/50
504/504 [==============================] - 0s 303us/step - loss: 128.9731 - val_loss: 163.6841
Epoch 46/50
504/504 [==============================] - 0s 300us/step - loss: 127.0357 - val_loss: 162.7604
Epoch 47/50
504/504 [==============================] - 0s 295us/step - loss: 125.1836 - val_loss: 160.9466
Epoch 48/50
504/504 [==============================] - 0s 316us/step - loss: 123.5052 - val_loss: 159.7201
Epoch 49/50
504/504 [==============================] - 0s 303us/step - loss: 121.8792 - val_loss: 159.0258
Epoch 50/50
504/504 [============================

504/504 [==============================] - 0s 283us/step - loss: 742.3827 - val_loss: 672.2790
Epoch 17/50
504/504 [==============================] - 0s 293us/step - loss: 595.5718 - val_loss: 534.5314
Epoch 18/50
504/504 [==============================] - 0s 311us/step - loss: 470.9218 - val_loss: 414.0790
Epoch 19/50
504/504 [==============================] - 0s 317us/step - loss: 366.1552 - val_loss: 328.8728
Epoch 20/50
504/504 [==============================] - 0s 302us/step - loss: 295.7618 - val_loss: 270.9845
Epoch 21/50
504/504 [==============================] - 0s 313us/step - loss: 248.3417 - val_loss: 235.5029
Epoch 22/50
504/504 [==============================] - 0s 291us/step - loss: 222.1110 - val_loss: 212.8085
Epoch 23/50
504/504 [==============================] - 0s 309us/step - loss: 204.1797 - val_loss: 200.3048
Epoch 24/50
504/504 [==============================] - 0s 290us/step - loss: 193.0462 - val_loss: 189.9060
Epoch 25/50
504/504 [============================

504/504 [==============================] - 0s 298us/step - loss: 156.2178 - val_loss: 162.2851
Epoch 43/50
504/504 [==============================] - 0s 316us/step - loss: 153.0013 - val_loss: 159.0798
Epoch 44/50
504/504 [==============================] - 0s 298us/step - loss: 149.9614 - val_loss: 155.9040
Epoch 45/50
504/504 [==============================] - 0s 300us/step - loss: 146.7847 - val_loss: 153.1049
Epoch 46/50
504/504 [==============================] - 0s 318us/step - loss: 143.6523 - val_loss: 150.4987
Epoch 47/50
504/504 [==============================] - 0s 324us/step - loss: 140.4231 - val_loss: 146.9772
Epoch 48/50
504/504 [==============================] - 0s 316us/step - loss: 137.6739 - val_loss: 144.2467
Epoch 49/50
504/504 [==============================] - 0s 319us/step - loss: 134.2069 - val_loss: 143.3229
Epoch 50/50
504/504 [==============================] - 0s 301us/step - loss: 130.7734 - val_loss: 139.7716
The mean MSE is 149.0182809084485
The standard de

In [49]:
print('The mean MSE is',np.mean(loss_list))
print('The standard deviation of MSE is',np.std(loss_list))

The mean MSE is 149.0182809084485
The standard deviation of MSE is 14.707290082278753


This is the best model so far with the lowest mean MSE. Increasing the number of layers helped.

<hr>

Copyright &copy; 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).